In [64]:

from langchain_pymupdf4llm import PyMuPDF4LLMLoader as pdfloader
from langchain_community.document_loaders.parsers import RapidOCRBlobParser


file_path = "aapl.pdf"


loader = pdfloader(file_path=file_path,mode='single',
                   pages_delimiter="/n-----PAGE ENDS HERE THIS IS A DELIMTER-----/n")

data = loader.load()

In [65]:

loader = pdfloader(file_path=file_path,mode='single',
                   pages_delimiter="/n-----PAGE ENDS HERE THIS IS A DELIMTER-----/n")

split_data = loader.load_and_split()

In [ ]:
from langchain_pymupdf4llm import PyMuPDF4LLMLoader as pdfloader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter, MarkdownTextSplitter, TextSplitter
from typing import List

In [50]:
def chunk_file(data):
    chunker = RecursiveCharacterTextSplitter(chunk_size=300)
    docs = chunker.create_documents(data)

    return docs


In [72]:
len(data[0].page_content)

433218

In [80]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200,
    chunk_overlap=200,
    separators=["\n\n", "\n", ".", " ", ""]
)
docs = splitter.create_documents([data[0].page_content])

In [82]:
len(docs)

503

In [85]:
chunker = SemanticChunker(OllamaEmbeddings(model='qwen3:0.6b'))
semantic_docs = chunker.create_documents([data[0].page_content])

In [87]:
len(semantic_docs)

88

In [89]:
from concurrent.futures import ThreadPoolExecutor
def chunk_segment(segment):
    chunker = SemanticChunker(OllamaEmbeddings(model="qwen3:0.6b"))
    return chunker.create_documents([segment])

# Split the input into 4 parts
full_text = data[0].page_content
segments = [full_text[i::4] for i in range(4)]

# Run in parallel
with ThreadPoolExecutor(max_workers=4) as executor:
    all_chunks = list(executor.map(chunk_segment, segments))

KeyboardInterrupt: 

In [ ]:
import threading